# Prefect (Continuación)

Ya vimos como crear `task`, `flow` y como ejecutar (`run`) cada uno. 

También vimos como podemos visualizar la ejecución de los mismos en el dashboard de `Prefect`.

Ahora vamos a ver como podemos programar la ejecución.

## **Definiciones en Prefect: Work Pools, Workers y Deployments**

### **Work Pools**:

Los **Work Pools** en `Prefect` sirven como un puente entre la capa de orquestación de `Prefect` y la infraestructura donde se ejecutan los flujos de trabajo (**flows**). Los **Work Pools** permiten gestionar de forma dinámica la provisión y configuración de infraestructura, optimizando la ejecución de tareas en diferentes entornos.
 

Un `Work Pool` define cómo y dónde se ejecutarán los flujos de trabajo (`flows`) y las tareas (`tasks`).

#### **Características de Work Pools**:
- **Abstracción de Infraestructura**: Permiten abstraer las configuraciones de ejecución, facilitando la implementación en diferentes entornos (local, nubes, contenedores, etc.).
- **Control de Concurrencia**: Gestionan cuántas tareas pueden ejecutarse simultáneamente.
- **Configuración Dinámica**: Los Work Pools se pueden configurar de manera dinámica para cambiar los entornos de ejecución sin modificar el código del flujo.
- **Provisionamiento dinámico**: Configuran infraestructuras dinámicas para evitar procesos inactivos y costosos. Ideal para flujos con requisitos de infraestructura elevados y que se ejecutan esporádicamente.
- **Configuración predeterminada**: Puedes establecer configuraciones de infraestructura predeterminadas que los trabajos heredan automáticamente, aunque pueden ser sobrescritas.
- **Interfaces opinadas**: Los equipos de plataforma pueden usar Work Pools para exponer interfaces predefinidas y controladas hacia la infraestructura bajo su supervisión.
- **Colas de trabajo**: Los Work Pools permiten priorizar o limitar las ejecuciones de los flujos mediante el uso de colas de trabajo (**work queues**).

### **Configuración de Work Pools**:

Puedes controlar diferentes aspectos del comportamiento de los Work Pools, como la cantidad de ejecuciones que pueden ejecutarse simultáneamente, y configurar su comportamiento usando:

- **Prefect UI**: Una interfaz gráfica para gestionar pools visualmente.
- **Prefect CLI**: Comandos de línea de comandos para crear y gestionar pools.
- **Prefect REST API**: APIs para la gestión de Work Pools desde otros sistemas.
- **Terraform Provider**: Gestión de Work Pools mediante **Terraform**.

### **Tipos de Work Pools soportados**:

| **Tipo de Infraestructura**             | **Descripción**                                                                                 |
| --------------------------------------- | ----------------------------------------------------------------------------------------------- |
| **Process**                             | Ejecuta flujos como subprocesos en un worker, ideal para ejecución local.                        |
| **AWS ECS (Elastic Container Service)** | Ejecuta flujos en contenedores en AWS ECS, compatible con EC2 y Fargate.                         |
| **Azure Container Instances**           | Ejecuta flujos en contenedores dentro de Azure Container Instances.                              |
| **Docker**                              | Ejecuta flujos dentro de contenedores Docker.                                                    |
| **Google Cloud Run**                    | Ejecuta flujos en contenedores dentro de Google Cloud Run.                                       |
| **Google Vertex AI**                    | Ejecuta flujos en contenedores en Google Vertex AI.                                              |
| **Kubernetes**                          | Ejecuta flujos dentro de un cluster de Kubernetes.                                               |
| **AWS ECS - Push**                      | Envía ejecuciones a AWS ECS directamente, sin necesidad de un Prefect worker.                    |
| **Azure Container Instances - Push**    | Envía ejecuciones a Azure Container Instances directamente, sin necesidad de un Prefect worker.  |
| **Prefect Managed**                     | Ejecuta flujos en infraestructuras gestionadas por Prefect.                                       |

### **Crear un Work Pool**:


1. Para crear un **Work Pool** usando la CLI de Prefect, utiliza el siguiente comando:

```bash
prefect work-pool create [OPTIONS] NAME
```

**NAME** es un nombre único obligatorio para el Work Pool.

#### **Opciones**:

- **--paused**: Si se proporciona, el Work Pool se crea en un estado pausado.
- **--type**: El tipo de infraestructura que puede ejecutar las ejecuciones desde este Work Pool.
- **--set-as-default**: Establece el Work Pool creado como el predeterminado local para el despliegue.
- **--base-job-template**: Ruta a un archivo JSON que contiene la plantilla base del trabajo a usar.

2. Interfaz gráfica
---

### **¿Qué son los Workers?**

Los **Workers** son servicios ligeros de polling que recuperan ejecuciones programadas de un **Work Pool** y las ejecutan. 

Cada worker tiene un tipo que corresponde al entorno de ejecución al cual debe enviar las ejecuciones de flujo (**flow runs**). 

Los workers solo pueden extraer trabajo de **work pools** que coincidan con su tipo. 

Como resultado, cuando los despliegues se asignan a un **work pool**, sabes en qué entorno se ejecutarán las ejecuciones de flujo programadas.

#### **Características principales de los Workers**:

- Los workers extraen trabajo de uno o más **work queues** dentro de un **work pool**.
- Cada tipo de worker corresponde a un entorno de ejecución específico, y los workers solo pueden extraer trabajo de pools que coincidan con su tipo.
- Cuando se asignan despliegues a un **work pool**, sabes en qué entorno se ejecutarán las ejecuciones de flujo programadas.

---

### **Tipos de Workers soportados**:

| **Tipo de Worker**                   | **Descripción**                                          | **Paquete Requerido**       |
| ------------------------------------ | -------------------------------------------------------- | --------------------------- |
| **process**                          | Ejecuta flujos como subprocesos.                         | N/A                         |
| **kubernetes**                       | Ejecuta flujos como trabajos de Kubernetes.              | `prefect-kubernetes`        |
| **docker**                           | Ejecuta flujos dentro de contenedores Docker.            | `prefect-docker`            |
| **ecs**                              | Ejecuta flujos como tareas de ECS.                       | `prefect-aws`               |
| **cloud-run**                        | Ejecuta flujos como trabajos de Google Cloud Run.        | `prefect-gcp`               |
| **vertex-ai**                        | Ejecuta flujos como trabajos de Google Cloud Vertex AI.  | `prefect-gcp`               |
| **azure-container-instance**         | Ejecuta flujos dentro de contenedores de ACI.            | `prefect-azure`             |

Si no ves un tipo de worker que se ajuste a tus necesidades, considera desarrollar un nuevo tipo de worker.

---

### **Opciones de Configuración de Workers**:

Cuando inicias un worker, puedes especificar los siguientes parámetros de configuración:

- `--name, -n`: Nombre que se le asignará al worker. Si no se proporciona, se genera un nombre único.
- `--pool, -p`: El **Work Pool** del cual el worker extraerá trabajo.
- `--work-queue, -q`: Uno o más nombres de **work queues** de los que el worker extraerá trabajo. Si no se especifica, el worker extrae de todas las colas en el pool.
- `--type, -t`: Tipo de worker a iniciar. Si no se proporciona, se infiere a partir del work pool.
- `--prefetch-seconds`: Tiempo antes de la hora de inicio programada para comenzar la ejecución del flujo. El valor predeterminado es el de `PREFECT_WORKER_PREFETCH_SECONDS`.
- `--run-once`: Ejecuta la encuesta del worker solo una vez. De manera predeterminada, el worker se ejecuta indefinidamente.
- `--limit, -l`: El número máximo de ejecuciones de flujos que el worker puede iniciar simultáneamente.
- `--with-healthcheck`: Inicia un servidor de **healthcheck** para el worker.
- `--install-policy`: Política de instalación para usar workers desde paquetes de integración de Prefect.

---

### **Ejemplo: Iniciar un Worker**:

Para iniciar un worker, utiliza el comando de la CLI de Prefect:

```bash
prefect worker start -p [nombre del work pool]
```

Por ejemplo:

```bash
prefect worker start -p "my-pool"
```

Salida esperada:

```bash
Discovered worker type 'process' for work pool 'my-pool'.
Worker 'ProcessWorker 65716280-96f8-420b-9300-7e94417f2673' started!
```

Además, puedes limitar el número de ejecuciones simultáneas de flujos con la opción `--limit`. Por ejemplo, para limitar un worker a cinco ejecuciones concurrentes:

```bash
prefect worker start --pool "my-pool" --limit 5
```

---

### ¿Qué es un `Deploy`?

Un `Deploy` en Prefect es una representación en el servidor de un flujo de trabajo (*flow*). 

Un `Deploy` almacena la metadata necesaria para la orquestación remota, incluyendo cuándo, dónde y cómo debe ejecutarse un flujo.

### Atributos principales de un despliegue:

- **Punto de entrada (*entrypoint*) del flujo**: Ruta hacia la función que define el flujo.
- **Programación o disparadores**: Reglas opcionales para programar o activar el flujo.
- **Etiquetas**: Etiquetas de texto opcionales para organizar los despliegues.

### Esquema del `Deploy`

El despliegue se define por su **nombre**, un **ID de flujo** y el **punto de entrada**. Opcionalmente, puede incluir:

- **Parámetros**: Valores predeterminados que se pasan al flujo en cada ejecución.
- **Programaciones**: Reglas de cronograma como expresiones cron o intervalos.
- **Etiquetas**: Para organizar y rastrear los despliegues.

### Creación del `Deploy`

Para crear un `deploy` en Prefect, tienes varias opciones:

1. **Usando un archivo `prefect.yaml`**:
   - Crea un archivo `prefect.yaml` en la raíz de tu proyecto.
   - Define las configuraciones del despliegue, los pasos de construcción y envío (*build* y *push*), y los pasos de obtención (*pull*).
   - Usa el comando `prefect deploy` para crear el despliegue.

2. **Usando un script en Python**:
   - Usa el método `flow.deploy()` dentro de tu código en Python.

3. **CLI interactiva**:
   - Ejecuta `prefect deploy` para seguir un proceso guiado de creación de despliegue.

### Puntos clave sobre los despliegues:

- Almacenan los metadatos para la orquestación remota, incluyendo cuándo, dónde y cómo debe ejecutarse un flujo de trabajo.
- Los atributos incluyen el punto de entrada (*entrypoint*) del flujo, el cronograma/disparador (*schedule/trigger*), etiquetas (*tags*), y más.
- Los despliegues permiten la gestión remota de ejecuciones de flujo, programación y aprovisionamiento de infraestructura.

### Para la integración con CI/CD:

- Muchas organizaciones utilizan procesos de CI/CD para gestionar despliegues en Prefect.
- Esto se puede combinar con las características de despliegue de Prefect para gestionar de manera eficiente actualizaciones del código del flujo, cambios de programación y compilaciones de contenedores.

Recuerda que la biblioteca `prefect-docker` es necesaria con frecuencia para crear despliegues que envían ejecuciones a la mayoría de los tipos de infraestructura de *work pools* de Prefect. Para despliegues en Kubernetes, se usa comúnmente la biblioteca `prefect-kubernetes`.

### Ejecución de `Deploy`

Un despliegue puede ser ejecutado de diversas maneras:

- **Manual**: A través de la `Prefect` CLI o SDK. `prefect deployment run 'flow-name/deployment-name'`
- **Programado**: Según una configuración previa, como cron.
- **Externo**: Utilizando APIs, eventos o sistemas externos para activar los flujos.

`Prefect` monitorea las ejecuciones y captura su estado para observabilidad y auditoría.

### Importancia del `Deploy`


Un despliegue separa el código del flujo de la orquestación, lo que permite ejecutar flujos en diferentes entornos sin que el código viva en el servidor de `Prefect`, manteniendo la separación entre tus activos propietarios y el backend de `Prefect`.

---


### **Diferencias entre Work Pools, Workers y Deployments**:

- **Work Pool**: Define el entorno donde se ejecutarán los flujos y tareas. Agrupa múltiples Workers.
- **Worker**: Unidad encargada de ejecutar las tareas individuales en el entorno especificado por el Work Pool.
- **Deployment**: Paquete que contiene el código del flujo, la configuración y los disparadores para la ejecución del flujo.

---

### **Ejemplo de Flujo Completo**:

1. **Crear un Work Pool** para especificar el entorno de ejecución. 
2. **Iniciar un Worker** para ejecutar las tareas en ese `Work Pool`.
3. **Crear un deployment** que empaqueta el `flow` y lo programa para que se ejecute en el `Work Pool`.
4. **Ejecutar el deployment**

---
